In [24]:
DL = '/Users/damiansp/Downloads'

In [25]:
!rm $DL/*

In [26]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [27]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADYEY', 'AMD', 'AMZN', 'ANET', 'APPH',
    'AXON', 'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG',
    'DKNG', 'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT',
    'FUV', 'FVRR', 'GH', 'GMED', 'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD',
    'LULU', 'MDB', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA',
    'NYT', 'OKTA', 'OPEN', 'PANW', 'PINS', 'PTON', 'QDEL', 'RBLX',
    'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP', 'SNOW',
    'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO',
    'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [28]:
!ls $DL

Portfolio_Positions_Jun-10-2022.csv Positions.csv


In [29]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [30]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [31]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADYEY
AMD


### E*Trade

In [32]:
DONGMEI = False

In [33]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,500.86
ABNB,--
ADYEY,2834.06
AMD,1611.9399999999998
AMZN,2193


In [34]:
#et_df.drop(index=['98986CONT'], inplace=True)
#t_df.head()

In [36]:
et_df.loc[et_df.ET == '--']
#et_df.loc['ABNB', 'ET'] = 108.91

In [37]:
check_symbols(et_df)

All clear


In [38]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,501
ABNB,109
ADYEY,2834
AMD,1612


### Fidelity

In [39]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Jun-10-2022.csv'

In [40]:
!ls $DL

Portfolio_Positions_Jun-10-2022.csv Positions.csv


In [41]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-02", "-03")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,3951.86,$1.00,$0.00,$3951.86,NaN,NaN,NaN,NaN,10.97%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.00,$137.13,-$5.51,$2194.08,-$88.16,-3.87%,-$230.08,-9.50%,6.09%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,3.00,$250.43,-$6.68,$751.29,-$20.04,-2.60%,-$16.31,-2.13%,2.08%,$767.60,$255.87,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.00,$108.91,-$6.81,$326.73,-$20.43,-5.89%,-$101.52,-23.71%,0.91%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,1.00,$14.03,-$1.068,$14.03,-$1.07,-7.08%,-$1.06,-7.03%,0.04%,$15.09,$15.09,Cash


In [42]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2194
2,ROLLOVER IRA,ABMD,751
3,ROLLOVER IRA,ABNB,327
4,ROLLOVER IRA,ADYEY,14
5,ROLLOVER IRA,AMD,1233


In [43]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2194
ABMD,751
ABNB,327
ADYEY,14
AMD,1233


In [44]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [45]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,2194,137,0
ABMD,501,751,250,0
ABNB,109,327,0,0
ADYEY,2834,14,1740,0
AMD,1612,1233,1422,284


In [46]:
out.to_csv(f'{DL}/clean.csv')

In [47]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
mw.csv


### MarketWatch

In [9]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

mv: rename /Users/damiansp/Downloads/Holdings - Damian Satterthwaite-Phillips.csv to /Users/damiansp/Downloads/Holdings - Damian Satterthwaite-Phillips(0).csv: No such file or directory
Portfolio_Positions_Jun-06-2022.csv Positions.csv


In [13]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_1,Value_2,Value_3
AAPL,727,436,1599
ABMD,1049,787,0
ABNB,719,599,1198
ADYEY,1117,3011,1381
AMD,1276,2339,1594


In [14]:
check_symbols(mw)

All clear


In [15]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,727,436,1599
ABMD,1049,787,0
ABNB,719,599,1198
ADYEY,1117,3011,1381
AMD,1276,2339,1594


In [16]:
mw.to_csv(f'{DL}/mw.csv')

In [17]:
!open $DL/mw.csv

In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*